## Health Survey Risk Factors
source: https://www.kaggle.com/datasets/cdc/behavioral-risk-factor-surveillance-system

In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

In [30]:
df=pd.read_csv('brfss2013.csv', encoding= 'unicode_escape', low_memory=False)

In [21]:
df.shape

(491775, 312)

In [5]:
for i in range(len(df.columns)):
    if df.columns[i]=='genhlth':
        print('the index of the general health column:', i)

the index of the general health column: 18


## EDA

In [31]:
## select columns with general health and other predictors
df=df.iloc[:,18:]

In [32]:
## Check missing values
# print('There are %i nan in the dataframe' % df.isna().sum().sum())
df.isnull().sum()/df.shape[0] 

genhlth      0.004036
physhlth     0.022281
menthlth     0.017543
poorhlth     0.494440
hlthpln1     0.003872
               ...   
X_rfseat3    0.079185
X_flshot6    0.697634
X_pneumo2    0.709784
X_aidtst3    0.111016
X_age80      0.000022
Length: 312, dtype: float64

- <0.1 : 339282 * 124 69% row remain
- <0.15 : 250425 * 135 49% row remain
- <0.2 : 209095 * 140 42% row remain

In [46]:
mask=(df.isnull().sum()/df.shape[0])<0.1
features=df.columns[mask]

In [47]:
df1 = df[features]

In [49]:
df1.dropna(inplace=True)

/var/folders/tc/hjg8dsq54vzfrk5v6v96k32r0000gn/T/ipykernel_12258/3614008390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.dropna(inplace=True)


In [52]:
df1.reset_index(inplace=True)

In [55]:
OneHotEncoder(df1)

TypeError: __init__() takes 1 positional argument but 2 were given